In [14]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [15]:
start_date = '20230306'
end_date = '20230306'

In [23]:
query = f"""

with start_loc as (
select 
    date_format(date_parse(yyyymmdd,'%Y%m%d'), '%Y-%m-%d') yyyymmdd,
    from_unixtime(cast(updated_epoch as double) /1000) order_start_time,
    order_id,
    city_name,
    service_obj_service_name service,
    -- estimate_id,
    ride_distance,
    captain_location_longitude start_long,
    captain_location_latitude start_lat,
    updated_epoch
    
from orders.order_logs_immutable
where 
    yyyymmdd BETWEEN '{start_date}' and '{end_date}' 
    and city_name is not null
    and service_obj_service_name in ('Link','Auto')
    
    and order_status = 'started'
    and event_type = 'started'
    -- and order_id = '640558e3c1cab2572ae5b5c6'
    -- and captain_id ='61ab4b1ca1f67139be985074'
),

drop_loc as (
select 
    date_format(date_parse(yyyymmdd,'%Y%m%d'), '%Y-%m-%d') yyyymmdd,
    from_unixtime(cast(updated_epoch as double) /1000) order_drop_time,
    order_id,
    city_name,
    service_obj_service_name service,
    -- estimate_id,
    ride_distance,
    distance_final_distance final_distance,
    captain_location_longitude drop_long,
    captain_location_latitude drop_lat,
    updated_epoch
    
from orders.order_logs_immutable
where 
    yyyymmdd BETWEEN '{start_date}' and '{end_date}' 
    and city_name is not null
    and service_obj_service_name in ('Link','Auto')
    
    and order_status = 'dropped'
    and event_type = 'dropped'
    -- and order_id = '640558e3c1cab2572ae5b5c6'
    -- and captain_id ='61ab4b1ca1f67139be985074'
)

select 
    start_loc.yyyymmdd,
    start_loc.service,
    start_loc.city_name,
    start_loc.order_id,
    
    start_loc.start_lat,
    drop_loc.drop_lat,
    abs(start_loc.start_lat - drop_loc.drop_lat) lat_diff,
    
    start_loc.start_long,
    drop_loc.drop_long,
    abs(start_loc.start_long - drop_loc.drop_long) long_diff,
    
    order_start_time,
    order_drop_time,
    (order_drop_time - order_start_time) time_diff,

    start_loc.ride_distance,
    drop_loc.final_distance
    
from start_loc 
join drop_loc on start_loc.yyyymmdd = drop_loc.yyyymmdd and start_loc.city_name = drop_loc.city_name and start_loc.order_id = drop_loc.order_id
order by 15 desc

"""
df_code = pd.read_sql(query, conn) 

In [24]:
df_code

,yyyymmdd,service,city_name,order_id,start_lat,drop_lat,lat_diff,start_long,drop_long,long_diff,order_start_time,order_drop_time,time_diff,ride_distance,final_distance
0,2023-03-06,Link,Hyderabad,6405a2dee6f11213438d39aa,17.516869,17.320965,1.959047e-01,78.438986,78.568672,1.296857e-01,2023-03-06 08:36:47.579,2023-03-06 09:40:42.541,0 01:03:54.962,31.73,4072.9586
1,2023-03-06,Link,Chennai,64055c3301b8b86aa8bc6559,13.018864,13.018864,7.797852e-08,80.169240,80.169243,3.358887e-06,2023-03-06 03:31:16.340,2023-03-06 03:31:23.530,0 00:00:07.190,1.40,3226.5890
2,2023-03-06,Link,Ahmedabad,640553db275d6f525c94d830,23.042059,23.077387,3.532796e-02,72.522234,72.589432,6.719786e-02,2023-03-06 02:52:22.227,2023-03-06 03:08:54.784,0 00:16:32.557,9.99,3003.7610
3,2023-03-06,Link,Patna,6405d4b160804939c421946b,25.601551,25.595690,5.861726e-03,85.209518,85.176109,3.340909e-02,2023-03-06 12:22:17.771,2023-03-06 12:51:20.376,0 00:29:02.605,4.92,2096.4520
4,2023-03-06,Link,Varanasi,6405b73f3fc4ae02c371ff13,25.302928,25.319477,1.654948e-02,83.004626,82.996147,8.478644e-03,2023-03-06 09:57:44.206,2023-03-06 10:21:26.199,0 00:23:41.993,2.55,1973.4800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861011,2023-03-06,Link,Kolkata,6406195e5c0b8649aad1a1aa,22.532663,22.532663,4.533691e-08,88.320745,88.320747,2.375488e-06,2023-03-06 16:55:15.762,2023-03-06 16:55:33.862,0 00:00:18.100,3.56,0.0000
861012,2023-03-06,Auto,Bangalore,640617775c0b8649aad19ef0,12.999781,12.999781,2.450928e-07,77.611176,77.611176,2.628906e-07,2023-03-06 16:49:10.866,2023-03-06 16:49:19.605,0 00:00:08.739,8.43,0.0000
861013,2023-03-06,Link,Delhi,64061d97275d6f525c9630b1,28.636556,28.636557,3.253662e-07,77.114647,77.114647,2.116211e-07,2023-03-06 17:18:36.949,2023-03-06 17:18:47.207,0 00:00:10.258,4.62,0.0000
861014,2023-03-06,Link,Bangalore,6405afa7275d6f525c957ecd,12.942473,12.942473,2.115601e-07,77.616168,77.616165,2.638867e-06,2023-03-06 09:21:35.445,2023-03-06 09:22:01.420,0 00:00:25.975,7.41,0.0000
